# DeepLense Regression

A FastAI-based tool for performing regression on strong lensing images to predict axion mass density of galaxies.


In [1]:
import os
import numpy as np
from tqdm import tqdm
import argparse
import sys
# setting path
sys.path.append('../utils')

image_shape = (150, 150)


from utils.utils import dir_path
sys.path.append(r'\Users\91963\deepl\DeepLense-Regression\path_Image\images.npy')
sys.path.append(r'\Users\91963\deepl\DeepLense-Regression\path_Image\mass_densities.npy')

path_to_images="path_Image"
result_path="result_dir"
file_names = os.listdir(path_to_images)
files_num = len(file_names)
img_shape = np.load(os.path.join(path_to_images, file_names[0]), allow_pickle=True)[0].shape
print("image is = ")
print(img_shape)
# Create base mmep_map on the disk where we will be writting the files
images_mmep = np.memmap(os.path.join(result_path,'images_mmep.npy'),
                        dtype='int16',
                        mode='w+',
                        shape=(files_num,*img_shape))

masses_mmep = np.memmap(os.path.join(result_path,'masses_mmep.npy'),
                        dtype='float32',
                        mode='w+',
                        shape=(files_num,1))

# Index that shows which part of the file is being written to
w_index = 0
for file_name in tqdm(file_names):
    img =np.load(r'C:\Users\91963\deepl\DeepLense-Regression\path_Image\images.npy',allow_pickle=True) # np.load(os.path.join(path_to_images, file_name),allow_pickle=True)
    mass = np.load(r'C:\Users\91963\deepl\DeepLense-Regression\path_Image\mass_densities.npy',allow_pickle=True)
    images_mmep = img[w_index:w_index+1,:]
    masses_mmep = mass[w_index:w_index+1,:]
    w_index+=1
    print("the process is running")

image is = 
(22500,)


 50%|█████     | 1/2 [00:00<00:00,  1.33it/s]

the process is running


100%|██████████| 2/2 [00:02<00:00,  1.11s/it]

the process is running


In [2]:
images_mmep.shape

(1, 22500)

In [3]:
masses_mmep.shape

(1, 1)

In [4]:
from fastai.basics import *
from fastai.vision.all import *
from fastai.callback.all import *
import torch
from torchvision import transforms

from models.xresnet_hybrid import xresnet_hybrid101
from utils.utils import inv_standardize,standardize, file_path, dir_path
from utils.custom_activation_functions import Mish_layer
from utils.custom_loss_functions import root_mean_squared_error, mae_loss_wgtd
from data.custom_datasets import RegressionNumpyArrayDataset

import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from tqdm import tqdm
import warnings

matplotlib.use('Agg')
%matplotlib inline
warnings.filterwarnings('ignore')

## Load the Data

In [5]:
path_to_images = r'C:\Users\91963\deepl\DeepLense-Regression\path_Image\images.npy'
path_to_masses = r'C:\Users\91963\deepl\DeepLense-Regression\path_Image\mass_densities.npy'

image_shape = (1,150,150)
# Number of images
images_num = 25000
# Load the dataset
# Memmap loads images to RAM only when they are used
images = np.memmap(path_to_images,
                   dtype='uint16',
                   mode='r',
                   shape=(images_num,*image_shape))

labels = np.memmap(path_to_masses,
                   dtype='uint16',
                   mode='r',
                   shape=(images_num,1))

In [6]:
print(f'Shape of images: {images.shape}')
print(f'Shape of masses: {labels.shape}')

Shape of images: (25000, 1, 150, 150)
Shape of masses: (25000, 1)


## Split the data

In [7]:
np.random.seed(234)
num_of_images = labels.shape[0]
max_indx_of_train_images = int(num_of_images*0.15)
max_indx_of_valid_images = max_indx_of_train_images + int(num_of_images*0.1)
max_indx_num_of_test_images = max_indx_of_valid_images + int(num_of_images*0.05)
permutated_indx = np.random.permutation(num_of_images)
train_indx = permutated_indx[:max_indx_of_train_images]
valid_indx = permutated_indx[max_indx_of_train_images:max_indx_of_valid_images]
test_indx = permutated_indx[max_indx_of_valid_images:max_indx_num_of_test_images]

In [8]:
print(f'Number of images in train: {int(num_of_images*0.15)}')
print(f'Number of images in valid: {int(num_of_images*0.1)}')
print(f'Number of images in test: {int(num_of_images*0.05)}')

Number of images in train: 3750
Number of images in valid: 2500
Number of images in test: 1250


## Transforms

In [9]:
base_image_transforms = [
    transforms.Resize(25000, 1, 150, 150)
]
rotation_image_transofrms = [
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=(0,360))
]

## Create a Dataset

In [10]:
train_dataset = RegressionNumpyArrayDataset(images, labels, train_indx,
                                            )
valid_dataset = RegressionNumpyArrayDataset(images, labels, valid_indx,                                          
                                            )
test_dataset = RegressionNumpyArrayDataset(images, labels, test_indx,                                 
                                           )

## Create a DataLoader

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')
batch_size = 64
dls = DataLoaders.from_dsets(train_dataset,valid_dataset,batch_size=batch_size, device=device, num_workers=2)

Device: cuda


## my code model for testing the all code

## Model Architecture

In [12]:
torch.manual_seed(50)
model = xresnet_hybrid101(n_out=1, sa=True, act_cls=Mish_layer, c_in = 1,device=device)

## Create a Learner

In [13]:
learn = Learner(
    dls, 
    model,
    opt_func=ranger, 
    loss_func= root_mean_squared_error,  
    metrics=[mae_loss_wgtd],
    model_dir = ''
)

## Find a Learning Rate

In [14]:
learn.lr_find()

In [ ]:
num_of_epochs = 1
lr = 1e-2
learn.fit_one_cycle(num_of_epochs,lr,cbs=
                    [ShowGraphCallback,
                     SaveModelCallback(monitor='mae_loss_wgtd',fname='best_model')])

epoch,train_loss,valid_loss,mae_loss_wgtd,time


## Load the best model

In [ ]:
learn.load('best_model',device=device)
learn.model = learn.model.to(device)

## Get Predictions for the Test Dataset

In [ ]:
test_dl = DataLoader(test_dataset, batch_size=1,shuffle=False,device=device)
m_pred,m_true = learn.get_preds(dl=test_dl,reorder=False)

## Plot the results

In [ ]:
test_mae = mae_loss_wgtd(m_pred,m_true)
plt.figure(figsize=(6,6),dpi=100)
plt.scatter(m_true, m_pred,  color='black')
line = np.linspace(0, 6, 100)
plt.plot(line, line)
plt.xlabel('Observed mass')
plt.ylabel('Predicted mass')
plt.text(1,4, 'MAE: {:.4f}'.format(test_mae))